In [1]:
"|IMPORT PACKAGES|"
import numpy              as np
import pandas             as pd
import datetime
from   bokeh.plotting     import show, figure, output_file, save
from   bokeh.io           import show, output_notebook, curdoc, export_png
from   bokeh.models       import ColumnDataSource,LinearAxis, Range1d, NumeralTickFormatter, LabelSet, Label, BoxAnnotation, DatetimeTickFormatter, Text, Span
from   bokeh.models.tools import HoverTool
from   bokeh.models       import Arrow, NormalHead, OpenHead, VeeHead
from   bokeh.transform    import dodge
from   bokeh.layouts      import gridplot
from   bokeh.layouts      import column
from   datetime           import datetime as dt

In [2]:
"|IMPORT DATA|"
path = r'https://github.com/ncachanosky/research/blob/master/Economic%20Series/'
file = r'Resumen%20Estadistico%20-%20Argentina.xlsx?raw=true'
IO   = path + file

sheet = 'EMAE'

data = pd.read_excel(IO, sheet_name = sheet, usecols="A:E,H:K,N:O", engine='openpyxl') # Be patient...

data['VAR_EMAE_1993']    = data['VAR_EMAE_1993']   *100
data['VAR_EMAE_SA_1993'] = data['VAR_EMAE_SA_1993']*100

In [3]:
"|CHECK DATA|"
data

,PERIODO,EMAE_1993,VAR_EMAE_1993,EMAE_SA_1993,VAR_EMAE_SA_1993,EMAE_2004,VAR_EMAE_2004,EMAE_SA_2004,VAR_EMAE_SA_2004,EMAE_2015,VAR_EMAE_2015
0,1993-01-01,87.389791,NaN,97.881527,NaN,NaN,NaN,NaN,NaN,57.376266,NaN
1,1993-02-01,85.141646,NaN,95.267198,-267.091162,NaN,NaN,NaN,NaN,55.843797,-2.670912
2,1993-03-01,101.928041,NaN,97.324170,215.916087,NaN,NaN,NaN,NaN,57.049554,2.159161
3,1993-04-01,101.890240,NaN,98.822488,153.951276,NaN,NaN,NaN,NaN,57.927840,1.539513
4,1993-05-01,103.674974,NaN,99.886790,107.698301,NaN,NaN,NaN,NaN,58.551713,1.076983
...,...,...,...,...,...,...,...,...,...,...,...
335,2020-12-01,NaN,NaN,NaN,NaN,132.4,-0.024,136.1,0.003,92.854294,0.300000
336,2021-01-01,NaN,NaN,NaN,NaN,129.7,-0.020,139.0,0.021,94.804234,2.100000
337,2021-02-01,NaN,NaN,NaN,NaN,127.0,-0.022,138.9,-0.001,94.709430,-0.100000
338,2021-03-01,NaN,NaN,NaN,NaN,144.1,0.119,138.5,-0.003,94.425301,-0.300000


In [14]:
"|BUILD PLOT|"

cds = ColumnDataSource(data)

#BUILD FIGURE 1
f1 = figure(title = "EL HUB ECONÓMICO | ESTIMADOR MENSUAL DE ACTIVIDAD ECONOMICA (EMAE)",
           y_range      = (0, 300),
           x_axis_type  = "datetime",
           plot_height  = 400,
           plot_width   = 700)

#AXIS 1 (LEFT)
p1 = f1.line('PERIODO', 'EMAE_1993'   , color='blue', line_alpha=0.25, width=2, legend_label='EMAE (1993=100)'   , muted_alpha=0.1, source=cds)
p2 = f1.line('PERIODO', 'EMAE_SA_1993', color='blue', line_alpha=0.75, width=2, legend_label='EMAE SA (1993=100)', muted_alpha=0.1, source=cds)
p3 = f1.line('PERIODO', 'EMAE_2004'   , color='red' , line_alpha=0.25, width=2, legend_label='EMAE (2004=100)'   , muted_alpha=0.1, source=cds)
p4 = f1.line('PERIODO', 'EMAE_SA_2004', color='red' , line_alpha=0.75, width=2, legend_label='EMAE SA (2004=100)', muted_alpha=0.1, source=cds)


f1.add_tools(HoverTool(renderers=[p2], tooltips = [("EMAE (1993=100)"  , "@EMAE_SA_1993{0.0}")], mode="vline"))
f1.add_tools(HoverTool(renderers=[p4], tooltips = [("EMAE (2004=100)"  , "@EMAE_SA_2004{0.0}")], mode="vline"))


#BUILD FIGURE 2
f2 = figure(title = "EL HUB ECONÓMICO | ESTIMADOR MENSUAL DE ACTIVIDAD ECONOMICA (EMAE) | EMPALME",
           y_range      = (0, 150),
           x_axis_type  ='datetime',
           plot_height  = 400,
           plot_width   = 700)

p5 = f2.line('PERIODO', 'EMAE_2015'   , color='blue', line_alpha=0.75, width=2, legend_label='EMAE SA (2015=100)' , muted_alpha=0.1, source=cds)

f2.add_tools(HoverTool(renderers=[p5], tooltips = [("EMAE (2015=100)", "@EMAE_2015")], mode="vline"))

#ADD NOTES
#Presidents' shades and names
Menem     = BoxAnnotation(left=dt(1989, 6, 8).timestamp()*1000, right=dt(1999,12,10).timestamp()*1000, line_color="black", fill_color="blue"  , fill_alpha=0.1)
DeLaRua   = BoxAnnotation(left=dt(1999,12,10).timestamp()*1000, right=dt(2001,12,20).timestamp()*1000, line_color="black", fill_color="red"   , fill_alpha=0.1)
Duhalde   = BoxAnnotation(left=dt(2002, 1, 2).timestamp()*1000, right=dt(2003, 5,25).timestamp()*1000, line_color="black", fill_color="blue"  , fill_alpha=0.1)
NKirchner = BoxAnnotation(left=dt(2003, 5,25).timestamp()*1000, right=dt(2007,12,10).timestamp()*1000, line_color="black" ,fill_color="blue"  , fill_alpha=0.1)
CFK       = BoxAnnotation(left=dt(2007,12,10).timestamp()*1000, right=dt(2015,12,10).timestamp()*1000, line_color="black" ,fill_color="blue"  , fill_alpha=0.1)
Macri     = BoxAnnotation(left=dt(2015,12,10).timestamp()*1000, right=dt(2019,12,10).timestamp()*1000, line_color="black", fill_color="yellow", fill_alpha=0.1)
Fernandez = BoxAnnotation(left=dt(2019,12,10).timestamp()*1000, right=dt(2021, 4,30).timestamp()*1000, line_color="black", fill_color="blue"  , fill_alpha=0.1)

x1 = (dt(1993, 1, 1).timestamp()*1000 + dt(1999,12,10).timestamp()*1000) / 2
x2 = (dt(1999,12,10).timestamp()*1000 + dt(2001,12,20).timestamp()*1000) / 2
x3 = (dt(2002, 1, 2).timestamp()*1000 + dt(2003, 5,25).timestamp()*1000) / 2
x4 = (dt(2003, 5,25).timestamp()*1000 + dt(2007,12,10).timestamp()*1000) / 2
x5 = (dt(2007,12,10).timestamp()*1000 + dt(2015,12,10).timestamp()*1000) / 2
x6 = (dt(2015,12,10).timestamp()*1000 + dt(2019,12,10).timestamp()*1000) / 2
x7 = (dt(2019,12,10).timestamp()*1000 + dt(2021, 4,30).timestamp()*1000) / 2

Menem_N     = Label(x=x1, y=10, text="Menem"      , text_font_size="8pt", text_align="center", text_baseline="middle", angle_units="deg", angle= 0)
DeLaRua_N   = Label(x=x2, y=10, text="De la Rúa"  , text_font_size="8pt", text_align="left"  , text_baseline="middle", angle_units="deg", angle=90)
Duhalde_N   = Label(x=x3, y=10, text="Duhalde"    , text_font_size="8pt", text_align="left"  , text_baseline="middle", angle_units="deg", angle=90)
NKirchner_N = Label(x=x4, y=10, text="N. Kirchner", text_font_size="8pt", text_align="center", text_baseline="middle", angle_units="deg", angle= 0)
CFK_N       = Label(x=x5, y=10, text="C. Kirchner", text_font_size="8pt", text_align="center", text_baseline="middle", angle_units="deg", angle= 0)
Macri_N     = Label(x=x6, y=10, text="Macri"      , text_font_size="8pt", text_align="center", text_baseline="middle", angle_units="deg", angle= 0)
Fernandez_N = Label(x=x7, y=10, text="AF"         , text_font_size="8pt", text_align="center", text_baseline="middle", angle_units="deg", angle= 0)


f1.add_layout(Menem)
f1.add_layout(DeLaRua)
f1.add_layout(Duhalde)
f1.add_layout(NKirchner)
f1.add_layout(CFK)
f1.add_layout(Macri)
f1.add_layout(Fernandez)

f1.add_layout(Menem_N)
f1.add_layout(DeLaRua_N)
f1.add_layout(Duhalde_N)
f1.add_layout(NKirchner_N)
f1.add_layout(CFK_N)
f1.add_layout(Macri_N)
f1.add_layout(Fernandez_N)

f2.add_layout(Menem)
f2.add_layout(DeLaRua)
f2.add_layout(Duhalde)
f2.add_layout(NKirchner)
f2.add_layout(CFK)
f2.add_layout(Macri)
f2.add_layout(Fernandez)

f2.add_layout(Menem_N)
f2.add_layout(DeLaRua_N)
f2.add_layout(Duhalde_N)
f2.add_layout(NKirchner_N)
f2.add_layout(CFK_N)
f2.add_layout(Macri_N)
f2.add_layout(Fernandez_N)

#LEGEND
f1.xaxis[0].formatter = DatetimeTickFormatter(months='%b %Y')
f2.xaxis[0].formatter = DatetimeTickFormatter(months='%b %Y')

f1.legend.location     = "top_left"
f1.legend.orientation  = "vertical"
f1.legend.click_policy = "mute"

f2.legend.location     = "top_left"
f2.legend.orientation  = "vertical"
f2.legend.click_policy = "mute"

p = gridplot([[f1], [f2]], plot_width=700, plot_height=400, toolbar_location='right')


show(p)

In [ ]:
"|EXPORT .PNG FILE|"

export_png(p, filename="emae.png")

In [ ]:
"|# CREATE HTML FILE|"

output_file(filename="emae.html", title="EMAE")
save(p)

In [15]:
"|CREATE JSON FILE|"
import json
import bokeh.embed

from bokeh.embed import json_item

j = json.dumps(json_item(p, "emae"))
with open("emae.json", "w") as fp:
    json.dump(j, fp)